In [0]:
from pyspark import SparkConf,SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import Row,SQLContext
import sys
import requests
# create spark configuration
conf = SparkConf()
conf.setAppName("TwitterStreamApp")
# create spark context with the above configuration
# create the Streaming Context from the above spark context with interval size 2 seconds
ssc = StreamingContext(sc, 2)
# setting a checkpoint to allow RDD recovery
ssc.checkpoint("/csv")
# read data from port 9009
dataStream = ssc.socketTextStream("localhost",9009)

In [0]:
def aggregate_tags_count(new_values, total_sum):
	return sum(new_values) + (total_sum or 0)

In [0]:
def send_df_to_dashboard(df):
  # extract the hashtags from dataframe and convert them into array
  top_tags = [str(t.hashtag) for t in df.select("hashtag").collect()]
  # extract the counts from dataframe and convert them into array
  tags_count = [p.hashtag_count for p in df.select("hashtag_count").collect()]
  print(top_tags)
   print(tags_count)

If you want to paste code into IPython, try the %paste and %cpaste magic functions.

File "<command-881310835193571>" , line 7 
 print(tags_count) 
 ^ 
 IndentationError : unindent does not match any outer indentation level

In [0]:
def get_sql_context_instance(spark_context):
  if ('sqlContextSingletonInstance' not in globals()):
    globals()['sqlContextSingletonInstance'] = SQLContext(spark_context)
    return globals()['sqlContextSingletonInstance']

def process_rdd(time, rdd):
  print("----------- %s -----------" % str(time))
  try:
    # Get spark sql singleton context from the current context
    sql_context = get_sql_context_instance(rdd.context)
    # convert the RDD to Row RDD
    row_rdd = rdd.map(lambda w: Row(hashtag=w[0], hashtag_count=w[1]))
    # create a DF from the Row RDD
    hashtags_df = sql_context.createDataFrame(row_rdd)
    # Register the dataframe as table
    hashtags_df.registerTempTable("hashtags")
    # get the top 10 hashtags from the table using SQL and print them
    hashtag_counts_df = sql_context.sql("select hashtag, hashtag_count from hashtags order by hashtag_count desc limit 10")
    hashtag_counts_df.show()
    # call this method to prepare top 10 hashtags DF and send them
    send_df_to_dashboard(hashtag_counts_df)
  except:
    e = sys.exc_info()[0]
    print("Error: %s" % e)

In [0]:
# split each tweet into words
words = dataStream.flatMap(lambda line: line.split(" "))
# filter the words to get only hashtags, then map each hashtag to be a pair of (hashtag,1)
hashtags = words.filter(lambda w: '#' in w).map(lambda x: (x, 1))
# adding the count of each hashtag to its last count
tags_totals = hashtags.updateStateByKey(aggregate_tags_count)
# do processing for each RDD generated in each interval
tags_totals.foreachRDD(process_rdd)
# start the streaming computation
ssc.start()
# wait for the streaming to finish
ssc.awaitTermination()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-881310835193567> in <module> () 
 4 hashtags = words . filter ( lambda w : '#' in w ) . map ( lambda x : ( x , 1 ) ) 
 5 # adding the count of each hashtag to its last count 
 ----> 6 tags_totals = hashtags . updateStateByKey ( aggregate_tags_count ) 
 7 # do processing for each RDD generated in each interval 
 8 tags_totals . foreachRDD ( process_rdd ) 

 /databricks/spark/python/pyspark/streaming/dstream.pyc in updateStateByKey (self, updateFunc, numPartitions, initialRDD) 
 602 initialRDD._jrdd)
 603 else : 
 --> 604 dstream = self . _sc . _jvm . PythonStateDStream ( self . _jdstream . dstream ( ) , jreduceFunc ) 
 605 
 606 return DStream ( dstream . asJavaDStream ( ) , self . _ssc , self . _sc . serializer ) 

 /databricks/spark/python/pyspark/streaming/dstream.pyc in _jdstream (self) 
 641 
 642 jfunc = TransformFunction ( self . _sc , self . func , self . prev . _jrdd_deserializer ) 
 --> 643 dstream = self . _sc . _jvm . PythonTransformedDStream ( self . prev . _jdstream . dstream ( ) , jfunc ) 
 644 self . _jdstream_val = dstream . asJavaDStream ( ) 
 645 return self . _jdstream_val 

 /databricks/spark/python/lib/py4j-0.10.6-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1426 answer = self . _gateway_client . send_command ( command ) 
 1427 return_value = get_return_value(
 -> 1428 answer, self._gateway_client, None, self._fqn)
 1429 
 1430 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.pyc in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.6-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 318 raise Py4JJavaError(
 319 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 320 format(target_id, ".", name), value)
 321 else : 
 322 raise Py4JError(

 Py4JJavaError : An error occurred while calling None.org.apache.spark.streaming.api.python.PythonTransformedDStream.
: java.lang.IllegalStateException: Adding new inputs, transformations, and output operations after starting a context is not supported
	at org.apache.spark.streaming.dstream.DStream.validateAtInit(DStream.scala:224)
	at org.apache.spark.streaming.dstream.DStream.<init>(DStream.scala:66)
	at org.apache.spark.streaming.api.python.PythonDStream.<init>(PythonDStream.scala:224)
	at org.apache.spark.streaming.api.python.PythonTransformedDStream.<init>(PythonDStream.scala:241)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.lang.reflect.Constructor.newInstance(Constructor.java:423)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:250)
	at py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
	at py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
	at py4j.GatewayConnection.run(GatewayConnection.java:226)
	at java.lang.Thread.run(Thread.java:748)